In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
# import the models for further classification experiments
from tensorflow.keras.applications import (
        vgg16,
        resnet50,
        mobilenet,
        inception_v3
    )


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
np.random.seed(1)
tf.random.set_seed(2)
# basic paths
train_dir = "/content/drive/MyDrive/Computer Science/6th semester/Projec Factory/model/dataset_v3/train"
test_dir = "/content/drive/MyDrive/Computer Science/6th semester/Projec Factory/model/dataset_v3/test"
val_dir = "/content/drive/MyDrive/Computer Science/6th semester/Projec Factory/model/dataset_v3/validation"

class_names = os.listdir(train_dir)
num_classes = len(class_names)

def get_dir_filenames(base_dir, name):
    dir = os.path.join(base_dir, name)

    return {"dir": dir, "file_names": os.listdir(dir)}


train = {name: get_dir_filenames(train_dir, name) for name in class_names}
val = {name: get_dir_filenames(val_dir, name) for name in class_names}

In [ ]:
train_len = sum(map(lambda x: len(x['file_names']), train.values()))
val_len = sum(map(lambda x: len(x['file_names']), val.values()))

print(f"Total training images: {train_len}")
print(f"Total validation images: {val_len}")

Total training images: 186
Total validation images: 29


In [ ]:
# init the models
# vgg_model = vgg16.VGG16(weights='imagenet')
# inception_model = inception_v3.InceptionV3(weights='imagenet')
# resnet_model = resnet50.ResNet50(weights='imagenet')
# mobilenet_model = mobilenet.MobileNet(weights='imagenet')

In [ ]:
# img_input = layers.Input(shape=(150, 150, 3))

# define a new model with random weights and 10 classes
# new_input = Input(shape=(640, 480, 3))
# model = VGG16(weights=None, input_tensor=new_input, classes=10)

model = inception_v3.InceptionV3(include_top=False, input_shape=(300, 300, 3), weights='imagenet')
# add new classifier layers
x = layers.Flatten()(model.layers[-1].output)
x = layers.Dense(512, activation='relu')(x)
output = layers.Dense(num_classes, activation='softmax')(x)
# define new model
model = Model(inputs=model.inputs, outputs=output)

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
# adam optimizer
model.compile(loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc'])

In [ ]:
# rms prop optimizer
# model.compile(loss='categorical_crossentropy',
#     optimizer=RMSprop(learning_rate=0.001),
#     metrics=['acc'])

In [ ]:
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)
# train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=6,
        class_mode='categorical',
        subset='training')


validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(300, 300),
        batch_size=2,
        class_mode='categorical')

Found 186 images belonging to 12 classes.
Found 29 images belonging to 12 classes.


In [ ]:
# fit model on the augmented dataset
model.fit_generator(train_generator, epochs=20, steps_per_epoch=31, validation_data=validation_generator)
# history = model.fit(
#       train_generator,
#       epochs=20,
#       validation_data=validation_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/20
31/31 [==============================] - 244s 7s/step - loss: 15.1332 - acc: 0.1022 - val_loss: 16102.5479 - val_acc: 0.1034
Epoch 2/20
31/31 [==============================] - 223s 7s/step - loss: 3.4909 - acc: 0.1505 - val_loss: 92.2293 - val_acc: 0.0690
Epoch 3/20
31/31 [==============================] - 222s 7s/step - loss: 2.8407 - acc: 0.1720 - val_loss: 268.0024 - val_acc: 0.1379
Epoch 4/20
31/31 [==============================] - 225s 7s/step - loss: 2.5751 - acc: 0.1720 - val_loss: 117.4932 - val_acc: 0.1379
Epoch 5/20
31/31 [==============================] - 231s 7s/step - loss: 2.4647 - acc: 0.2204 - val_loss: 2.8277 - val_acc: 0.1034
Epoch 6/20
31/31 [==============================] - 226s 7s/step - loss: 2.5565 - acc: 0.1935 - val_loss: 2.4917 - val_acc: 0.1379
Epoch 7/20
31/31 [==============================] - 226s 7s/step - loss: 2.4770 - acc: 0.1935 - val_loss: 2.4880 - val_acc: 0.1379
Epoch 8/20
31/31 [==============================] - 224s 7s/step - loss: 

In [ ]:
# Retrieve a list of accuracy results on training and validation data
# sets for each training epoch
acc = history.history['acc']
val_acc = history.history['val_acc']

# Retrieve a list of list results on training data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training accuracy per epoch
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')

plt.figure()

# Plot training loss per epoch
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')

NameError: ignored

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(300, 300),
        class_mode='categorical',
        batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)

logits = model.predict(test_generator,steps=nb_samples)
prediction = tf.nn.softmax(logits)
print(prediction)

Found 28 images belonging to 12 classes.
tf.Tensor(
[[0.0728964  0.19813186 0.0728964  0.07290405 0.07289641 0.0728964
  0.07289648 0.07289641 0.0728964  0.0728964  0.0728964  0.0728964 ]
 [0.07289543 0.19815032 0.07289543 0.07289543 0.07289543 0.07289543
  0.07289543 0.07289543 0.07289543 0.07289543 0.07289543 0.07289543]
 [0.07289543 0.19815032 0.07289543 0.07289543 0.07289543 0.07289543
  0.07289543 0.07289543 0.07289543 0.07289543 0.07289543 0.07289543]
 [0.07289543 0.19815032 0.07289543 0.07289543 0.07289543 0.07289543
  0.07289543 0.07289543 0.07289543 0.07289543 0.07289543 0.07289543]
 [0.0734852  0.18658228 0.07348502 0.07653432 0.07381904 0.07348505
  0.0743377  0.07414541 0.07360886 0.0735452  0.07348567 0.07348631]
 [0.07289543 0.19815032 0.07289543 0.07289543 0.07289543 0.07289543
  0.07289543 0.07289543 0.07289543 0.07289543 0.07289543 0.07289543]
 [0.08124015 0.09577233 0.07929728 0.08519217 0.08117463 0.08159412
  0.0820834  0.08125321 0.08577837 0.08166364 0.08180981 0.